# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import re
import time
import requests

from functools import wraps

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

In [3]:
def convert_seconds(time_apply, time_format=False):
    try:
        time_apply = float(time_apply)
    except ValueError:
        time_apply = 0
    if isinstance(time_apply, (int, float)):
        hrs = time_apply // 3600
        mns = time_apply % 3600
        sec = mns % 60
        time_string = ''
        if hrs:
            time_string = f'{hrs:.0f} час '
        if mns // 60 or hrs:
            time_string += f'{mns // 60:.0f} мин '
        if time_format:
            return f'{int(hrs)}:{int(mns // 60):02}:{int(sec):02}'
        return f'{time_string}{sec:.3f} сек'

## Задание 1

In [4]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {convert_seconds(end - start)}')
        return result

    return wrapper

## Задание 2

In [5]:
def logging(func):
    """
    Декоратор, который выводит параметры, с которыми была вызвана функция
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f'Функция вызвана с параметрами:\n{args}, {kwargs}')
        return func(*args, **kwargs)

    return wrapper

## Задание 3

In [6]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        wrapper.call_count += 1
        print(f"Функция '{func.__name__}' была вызвана {wrapper.call_count} раз(а).")
        return result

    wrapper.call_count = 0  # Инициализируем счетчик
    return wrapper

## Задание 4

In [7]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func,
    чьи аргументы args должны быть хешируемыми
    """
    cache = {}

    def func_get_memo(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]

    func_get_memo.cache = cache
    return func_get_memo

## Тестирование

In [8]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция вызвана с параметрами:
('whole',), {}
Время выполнения функции word_count: 3.578 сек
Функция 'word_count' была вызвана 1 раз(а).
Cлово whole встречается 176 раз


In [9]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [10]:
%%time
# измеряем время выполнения
fib(37)

CPU times: total: 2.72 s
Wall time: 2.72 s


24157817

In [11]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [12]:
%%time
# измеряем время выполнения
fib(37)

CPU times: total: 0 ns
Wall time: 0 ns


24157817

In [13]:
# проверка как работает счетчик вызовов функции

@logging
@counter
def fib(n):
    if n < 2:
        return n
    return fib(n - 2) + fib(n - 1)

fib(3)

Функция вызвана с параметрами:
(3,), {}
Функция вызвана с параметрами:
(1,), {}
Функция 'fib' была вызвана 1 раз(а).
Функция вызвана с параметрами:
(2,), {}
Функция вызвана с параметрами:
(0,), {}
Функция 'fib' была вызвана 2 раз(а).
Функция вызвана с параметрами:
(1,), {}
Функция 'fib' была вызвана 3 раз(а).
Функция 'fib' была вызвана 4 раз(а).
Функция 'fib' была вызвана 5 раз(а).


2